In [1]:
pwd

'/mmfs1/data/aglinska/BC-MRI-AE/Colab Notebooks'

In [2]:
%%time
from importlib import reload
#import helper_funcs;reload(helper_funcs2);from helper_funcs2 import *
import make_models2;reload(make_models2);from make_models2 import *
import pickle
from tqdm import tqdm

CPU times: user 2.37 s, sys: 3.61 s, total: 5.98 s
Wall time: 10.9 s


In [3]:
arr = np.load('/mmfs1/data/aglinska/BC-MRI-AE/Data/ABIDE-Anat-64iso.npz')
ABIDE_data = arr['data']
ABIDE_subs = arr['subs']
nsubs = ABIDE_data.shape[0]
[arr.shape for arr in [ABIDE_subs,ABIDE_data]]

aa = np.array([ABIDE_data[s,:,:,:].sum() for s in range(ABIDE_data.shape[0])])<5000
ABIDE_data = ABIDE_data[~aa,:,:,:]
ABIDE_subs = ABIDE_subs[~aa]
[arr.shape for arr in [ABIDE_subs,ABIDE_data]]

df = pd.read_csv('/mmfs1/data/aglinska/BC-MRI-AE/CSVs/ABIDE-legend.csv',header=0)
df = df.iloc[np.array([df['BIDS_ID'].values[s] in ABIDE_subs for s in range(len(df))])]
df.reset_index(inplace=True)

assert len(df)==len(ABIDE_subs),'dif lenghts'
assert all([df['BIDS_ID'][s]==ABIDE_subs[s] for s in range(len(df))]),'mismatch between df and goodsubs'

df.head(5)

,index,BIDS_ID,Anonymized ID,Subject Type,Visit,Days since enrollment,SubID,DxGroup,DSMIVTR,AgeAtScan,...,WISC4 Pic Con Scaled,WISC4 Matrix Scaled,WISC4 Dig Span Scaled,WISC4 Let Num Scaled,WISC4 Coding Scaled,WISC4 Sym Scaled,Age at MPRAGE,Current Med Status,Medication Name,Off stimulants at EPI scan?
0,0,sub-001,A00033348,PATIENT,session_1,8,51320,1,2.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,1,sub-002,A00033398,CONTROL,session_1,8,51373,2,0.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,2,sub-003,A00033352,PATIENT,session_1,8,51324,1,2.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,3,sub-004,A00033389,CONTROL,session_1,8,51364,2,0.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,4,sub-005,A00033354,PATIENT,session_1,8,51326,1,2.0,42.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [4]:
%%time 
# LOAD CVAE
latent_dim = 16;batch_size = 32;beta = 1;gamma = 100;disentangle = True
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(latent_dim=latent_dim,beta=beta, disentangle=disentangle, gamma=gamma, bias=True, batch_size = batch_size)
loss = list()    
fn = '/mmfs1/data/aglinska/tf_outputs/CVAE/Feb23_1551_D16B1G100_DTrue' # good for paper
loss = pickle.load(open(fn+'_loss.pickle','rb'))
cvae.load_weights(fn)

CPU times: user 2.41 s, sys: 1.39 s, total: 3.8 s
Wall time: 4.28 s


In [21]:
mses = list()
for i in range(14):
    data_batch = ABIDE_data[np.random.permutation(np.nonzero(df['DxGroup'].values==1)[0])[i*32:i*32+32],:,:,:]
    mse = ((data_batch-cvae.predict([data_batch,data_batch])[0][:,:,:,:,0])**2).mean()
    print(mse)
    mses.append(mse)

0.0039087986
0.0037922796
0.0034115503
0.0034304962
0.0036486753
0.0032719693
0.003532045
0.0034819553
0.0035871877
0.0036035122
0.00363411
0.0031975042
0.0033240605
0.003941023


In [24]:
np.array(mses).mean()

0.0035546545

In [7]:
latent_dim = 32
batch_size = 32
disentangle = False
gamma = 1

encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1), 
                                        latent_dim=32, 
                                        batch_size = batch_size, 
                                        disentangle=True,
                                        gamma=1,
                                        kernel_size = 3,
                                        filters = 32,
                                        intermediate_dim = 128+64,
                                        nlayers = 2,
                                        bias=True)
    

fn = '/mmfs1/data/aglinska/tf_outputs/VAE/20210917-1506-3'
vae.load_weights(fn)
loss = pickle.load(open(fn+'_loss.pickle','rb'))

In [15]:
len((np.nonzero(df['DxGroup'].values==1))[0])/3

14.6875

In [17]:
mses = list()
for i in range(14):
    data_batch = ABIDE_data[np.random.permutation(np.nonzero(df['DxGroup'].values==1)[0])[i*32:i*32+32],:,:,:]
    mse = ((data_batch-vae.predict(data_batch)[:,:,:,:,0])**2).mean()
    print(mse)
    mses.append(mse)

0.004850729
0.004538235
0.005419832
0.005045265
0.005077817
0.0050479984
0.0051961592
0.004990591
0.005075469
0.0049923453
0.004979443
0.005007255
0.005095245
0.00511528


In [19]:
np.array(mses).mean()

0.005030833